# VAEP 모델 비교: CatBoost vs XGBoost

이 노트북에서는 K-League 데이터를 사용하여 VAEP 점수를 계산하기 위한 두 가지 강력한 부스팅 모델인 CatBoost와 XGBoost의 성능을 비교합니다.

## 1. 환경 설정 및 데이터 로드

In [ ]:
import pandas as pd
import yaml
import os
import glob
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import socceraction.vaep.features as features
import socceraction.vaep.labels as labels

# 설정 로드
with open('../config.yaml', 'r') as f:
    config = yaml.safe_load(f)

SPADL_DIR = os.path.join('..', config['data']['spadl_output_dir'])
NB_PREV_ACTIONS = config['features']['nb_prev_actions']

def load_spadl_data():
    all_games = []
    files = glob.glob(os.path.join(SPADL_DIR, "game_*.csv"))
    print(f"Loading {len(files)} games for model comparison...")
    for f in files:
        df = pd.read_csv(f)
        all_games.append(df)
    return all_games

games = load_spadl_data()

## 2. Feature 및 Label 생성

In [ ]:
def get_feature_functions():
    return [
        features.actiontype_onehot,
        features.result_onehot,
        features.actiontype_result_onehot,
        features.bodypart_onehot,
        features.time,
        features.startlocation,
        features.endlocation,
        features.movement,
        features.space_delta,
        features.team,
    ]

def generate_dataset(games):
    X_list = []
    Y_list = []
    xfns = get_feature_functions()

    print("Generating features and labels...")
    for game in games:
        try:
            gamestates = features.gamestates(game, NB_PREV_ACTIONS)
            X_game = pd.concat([fn(gamestates) for fn in xfns], axis=1)
            
            Y_scores = labels.scores(game, nr_actions=10)
            Y_concedes = labels.concedes(game, nr_actions=10)
            Y_game = pd.concat([Y_scores, Y_concedes], axis=1)
            Y_game.columns = ['scores', 'concedes']
            
            X_list.append(X_game)
            Y_list.append(Y_game)
        except Exception as e:
            print(f"Skipping game due to error: {e}")
            continue

    if not X_list:
        raise ValueError("No data generated.")

    X = pd.concat(X_list).reset_index(drop=True)
    Y = pd.concat(Y_list).reset_index(drop=True)
    return X, Y

X, Y = generate_dataset(games)
print(f"Dataset shape: {X.shape}")

## 3. 모델 학습 및 평가

데이터를 학습 세트와 검증 세트로 나누고(80:20), 각 모델을 학습시킵니다.
- **CatBoost**: 범주형 변수 처리에 강점이 있으며 기본 설정으로도 좋은 성능을 보입니다.
- **XGBoost**: 빠르고 효율적이며 튜닝 시 높은 성능을 낼 수 있습니다.

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

models = {
    'CatBoost': CatBoostClassifier(verbose=0, random_state=42),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

results = {}

for name, model in models.items():
    print(f"Training {name}...")
    # Scoring Model
    model.fit(X_train, Y_train['scores'])
    pred_scores = model.predict_proba(X_val)[:, 1]
    auc_scores = roc_auc_score(Y_val['scores'], pred_scores)
    
    # Conceding Model
    model.fit(X_train, Y_train['concedes'])
    pred_concedes = model.predict_proba(X_val)[:, 1]
    auc_concedes = roc_auc_score(Y_val['concedes'], pred_concedes)
    
    results[name] = {
        'AUC (Scores)': auc_scores,
        'AUC (Concedes)': auc_concedes,
        'Average AUC': (auc_scores + auc_concedes) / 2,
        'pred_scores': pred_scores,
        'pred_concedes': pred_concedes
    }
    print(f"{name} Results: {results[name]}")

## 4. 결과 시각화 (ROC Curve)

In [ ]:
plt.figure(figsize=(12, 5))

# Scoring ROC
plt.subplot(1, 2, 1)
for name in models.keys():
    fpr, tpr, _ = roc_curve(Y_val['scores'], results[name]['pred_scores'])
    plt.plot(fpr, tpr, label=f"{name} (AUC={results[name]['AUC (Scores)']:.3f})")
plt.plot([0, 1], [0, 1], 'k--')
plt.title('ROC Curve - Scoring')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()

# Conceding ROC
plt.subplot(1, 2, 2)
for name in models.keys():
    fpr, tpr, _ = roc_curve(Y_val['concedes'], results[name]['pred_concedes'])
    plt.plot(fpr, tpr, label=f"{name} (AUC={results[name]['AUC (Concedes)']:.3f})")
plt.plot([0, 1], [0, 1], 'k--')
plt.title('ROC Curve - Conceding')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()

plt.tight_layout()
plt.savefig('../results/model_comparison_roc.png')
plt.show()

## 5. 결론

위의 ROC 커브와 AUC 점수를 통해 어떤 모델이 K-League 데이터셋에 더 적합한지 판단할 수 있습니다. 일반적으로 두 모델 모두 우수한 성능을 보이지만, 데이터 특성에 따라 미세한 차이가 발생할 수 있습니다.